In [ ]:
import random
import numpy as np
import pandas as pd
from shutil import copyfile
import skimage as sk
from skimage import util, transform
import os
from shutil import rmtree
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img 
from distutils.dir_util import copy_tree

# Leaf Image Augmentation

In [ ]:
# Rutas de Origen y Destino
data_path = "C:/Users/adoncel/Documents/Kaggle/cassava-leaf-disease-classification/"
data_augment_path = "C:/Users/adoncel/Documents/Kaggle/cassava-leaf-disease-classification-augment/"


# Etiquetas
train_csv_data_path = data_path+"train.csv"
label_json_data_path = data_path+"label_num_to_disease_map.json"
images_dir_data_path = data_path+"train_images"

train_csv = pd.read_csv(train_csv_data_path)
train_csv['label'] = train_csv['label'].astype('str')

classes = set(train_csv['label'])


# Rutas de Origen y Destno del Tipo de Datos (Train, Val, Test)
directory_root = os.path.join(data_path, 'train_images')
directory_destino = os.path.join(data_augment_path, 'train_images')


# Listar Todas las Imagenes
images = [os.path.join(directory_root, f) for f in os.listdir(directory_root) if os.path.isfile(os.path.join(directory_root, f))]


# Diferenciar imagenes por tipo de clase
images_class = {}
length_class = {}
train_csv['image_id'] = train_csv['image_id'].apply(lambda x: os.path.join(directory_root,x))
for c in classes:
    df_label = train_csv[train_csv['label']==c]
    images_class[c] = list(df_label['image_id'])
    length_class[c] = len(list(df_label['image_id']))


# Definir rango de aumentacion requerido para cada clase 
max_class = length_class[max(length_class, key=length_class.get)]
length_class = {c: np.round(max_class/length_class[c]) for c in length_class.keys()}


# Aumentar X por cada tipo de clase
for tipo in classes:
    N_AUGMENT = length_class[tipo]
    images_tipo = images_class[tipo]
    
    folder_path_aug = os.path.sep.join([directory_destino, str(tipo)])
    
    if(os.path.isdir(folder_path_aug)):
        rmtree(folder_path_aug)
        time.sleep(.300)
        os.mkdir(folder_path_aug)
    else:
        os.makedirs(folder_path_aug)

    # Generador de Imagenes
    datagen = ImageDataGenerator(rotation_range=360,
                                width_shift_range=0.1,
                                height_shift_range=0.1,
                                brightness_range=[0.5,0.9],
                                shear_range=25,
                                zoom_range=0.3,
                                channel_shift_range=0.1,
                                horizontal_flip=True,
                                vertical_flip=True,
                                rescale=1/255,
                                validation_split=0.15)

    # Bucle de generacion de Imagenes   
    for image_path in images_tipo:
        img = load_img(image_path)
        x = img_to_array(img) 
        
        # Inicializar contador
        i=0
        
        
        # Copy Original File
        copyfile(image_path, image_path.replace(directory_root, folder_path_aug))
        i += 1
              
        x = x.reshape((1, ) + x.shape)
        if i < N_AUGMENT:
            for batch in datagen.flow(x, batch_size = 1,#nt(N_AUGMENT), 
                                      save_to_dir = folder_path_aug,  
                                      save_prefix ='image', save_format ='jpeg'): 
                i += 1

                if i > N_AUGMENT: 
                    break